In [1]:
import torch
import torch.nn as nn
from math import ceil

In [2]:
base_model = [
    # expand_ration, channels, repeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

In [3]:
phi_values = {
    # tuple of (phi, value, resolution, drop_rate)
    'b0': (0, 224, 0.2), # alpha, beta, gamma i.e.: depth = alpha**phi
    'b1': (0.5, 240, 0.2),
    'b2': (1, 260, 0.3),
    'b3': (2, 300, 0.3),
    'b4': (3, 380, 0.3),
    'b5': (4, 456, 0.4),
    'b6': (5, 528, 0.5),
    'b7': (6, 600, 0.5),
}

In [4]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups=1):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups=groups,
            bias= False,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU() # SiLu <-> Swish
    
    def forward(self, x):
        return self.silu(self.bn(self.cnn(x)))

In [5]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # C x H x W -> C x 1 x 1 
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return x * self.se(x)

In [6]:
class InvertedResidualBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        expand_ratio,
        reduction=4, # SE
        survival_prob=0.8 # for stochastic depth
    ):
        super(InvertedResidualBlock, self).__init__()
        self.survival_prob = 0.8
        self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = in_channels * expand_ratio
        self.expand = in_channels != hidden_dim
        reduced_dim = int(in_channels / reduction)
        
        if self.expand:
            self.expand_conv = CNNBlock(
                in_channels, hidden_dim, kernel_size=3, stride=1, padding=1,
            )
        
        self.conv = nn.Sequential(
            CNNBlock(
                hidden_dim, hidden_dim, kernel_size, stride, padding, groups=hidden_dim,
            ),
            SqueezeExcitation(hidden_dim, reduced_dim),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        
    def stochastic_depth(self, x):
        if not self.training:
            return x
        
        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device= x.device) < self.survival_prob
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs
        if self.use_residual:
            return self.stochastic_depth(self.conv(x)) + inputs
        else:
            return self.conv(x)

In [7]:
class EfficientNet(nn.Module):
    def __init__(self, version, num_classes):
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes)
        )
        
    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depht_factor = alpha ** phi
        width_factor = beta ** phi
        return width_factor, depht_factor, drop_rate
    
    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1)]
        in_channels = channels
        
        for expand_ratio, channels, repeats, stride, kernel_size, in base_model:
            out_channels = 4 * ceil(int(channels * width_factor) / 4) 
            layers_repeats =  ceil(repeats * depth_factor)
            
            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        in_channels,
                        out_channels,
                        expand_ratio= expand_ratio,
                        stride= stride if layer == 0 else 1,
                        kernel_size = kernel_size,
                        padding = kernel_size // 2, # if k = 1: pad=0, k=3:pad:1, k=5:pad:2
                    )
                )
                in_channels = out_channels
                
        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0)
        )
        
        return nn.Sequential(*features)
    
    def forward(self, x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0], -1))

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
version = 'b0'
phi, res, drop_rate = phi_values[version]
num_examples, num_classes = 2, 4
x = torch.randn((num_examples, 3, res, res), device=device)
model = EfficientNet(
    version=version,
    num_classes=num_classes
).to(device)

print(model(x).shape)

torch.Size([2, 4])


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import sklearn
import matplotlib.pyplot as plt
import torchvision

In [10]:
imgSize = 224

# Transformations
# Training transformer
transformer = transforms.Compose([
    transforms.RandomRotation(degrees = 20),
    
    transforms.RandomHorizontalFlip(p = 0.3),
    transforms.RandomVerticalFlip(p = 0.3),
    
    transforms.Resize(size = (imgSize, imgSize), antialias = True),
    transforms.ToTensor(),
])

# Validation transformer
valTransformer = transforms.Compose([
    transforms.Resize(size = (imgSize, imgSize), antialias = True),
    transforms.ToTensor(),
])

In [11]:
# hyperparameters
BATCH_SIZE = 64
learning_rate = 0.0001

In [12]:
path = '/kaggle/input/melanoma-cancer-dataset/train'
valPath = '/kaggle/input/melanoma-cancer-dataset/test'

# Reading data
trainData = datasets.ImageFolder(root = path, transform = transformer)
testData = datasets.ImageFolder(root = valPath, transform = valTransformer)

train_loader = DataLoader(dataset=trainData,batch_size= BATCH_SIZE, shuffle= True, num_workers=4)
test_loader = DataLoader(dataset=testData, batch_size= BATCH_SIZE, shuffle= False, num_workers=4)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [14]:
def check_accuracy(loader, model, Train= True):
    if Train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    acc = float(num_correct/float(num_samples))
    print(f'{num_correct}/{num_samples} with accuracy {float(num_correct/float(num_samples))*100:.2f}')
    model.train()
    return acc

In [15]:
# Early Stopping
patience = 5
minDelta = 0.01
currentPatience = 0
bestLoss = float('inf')

In [16]:
epochs = 10
for epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(enumerate(train_loader), total= len(train_loader), leave= False)
    for batch_idx, (data, targets) in loop:
        # Send data to device
        data = data.to(device)
        targets = targets.to(device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        
        #log
        runningLoss += loss.item()
        
        #otimizer
        optimizer.step()
        
        # update progress bar
        loop.set_description(f'Epoch [{epoch}/{epochs}]')
        loop.set_postfix(loss = loss.item())
    
    trainLoss= runningLoss/len(train_loader)
    check_accuracy(train_loader, model, Train=True)
check_accuracy(test_loader, model, Train=False)

Checking accuracy on training data


9918/11879 with accuracy 83.49


Checking accuracy on training data


10178/11879 with accuracy 85.68


Checking accuracy on training data


...

Checking accuracy on training data


10535/11879 with accuracy 88.69


Checking accuracy on training data


10615/11879 with accuracy 89.36
Checking accuracy on test data
1798/2000 with accuracy 89.90


0.8990000486373901